In [5]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [ ]:
from tensorflow.keras.models import load_model
import pickle

model = load_model('ner_model.h5')

with open('x_tokenizer.pkl' , 'rb') as f:
  x_tokenizer = pickle.load(f)

with open('y_tokenizer.pkl' , 'rb') as f:
  y_tokenizer = pickle.load(f)

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
max_x_seq = 89

def predict_tags(text):
    # Tokenize the input text
    dic = {}
    text = text.lower()
    tokenized_text = x_tokenizer.texts_to_sequences([text])
    # Pad the tokenized text to the same length as training data
    padded_text = pad_sequences(tokenized_text, padding='post', maxlen=max_x_seq)

    # Make prediction
    prediction = model.predict(padded_text)
    predicted_tag_indices = np.argmax(prediction, axis=-1)[0]
    for i in range(len(padded_text[0])):
        idx2 = predicted_tag_indices[i]
        idx1 = padded_text[0][i]
        if idx1 != 0:
          dic[x_tokenizer.index_word[idx1]] = y_tokenizer.index_word[idx2]
    return dic


import asyncio
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi.responses import JSONResponse
from pydantic import BaseModel

app = FastAPI()

class StrInput(BaseModel):
  text : str


@app.get('/')
def home():
  return 'NOTHING HERE GO TO /predict'

@app.post("/predict")
def predict_ner(data : StrInput):
  return JSONResponse(predict_tags(data.text))


''' One important note I was running this code in colab so for colab I was using ngrok
if you are running this in your local machine then no need to use ngrok use uvicorn's standard method
to run it on localhost. The code below is useless if you are running it on your local machine.'''



nest_asyncio.apply()

# Set ngrok token
ngrok.set_auth_token("Use your authToken")

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

async def uvicorn_serve():
  config = uvicorn.Config(app, host="0.0.0.0", port=8000, loop="asyncio")
  server = uvicorn.Server(config)
  await server.serve()
# Schedule the server to run as a task in the existing event loop
loop = asyncio.get_event_loop()
loop.create_task(uvicorn_serve())

In [2]:
from pyngrok import ngrok

# Terminate any ngrok tunnels currently running
ngrok.kill()